In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("./data/230907_basic_table_for_analysis_cleaned.csv", index_col=0, low_memory=False)

In [5]:
# cleanning variables that might not be used in the model
df['date']= pd.to_datetime(df.date,format='%Y-%m-%d')
df['year'] = df['date'].dt.year.astype(int)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['actp'] = le.fit_transform(df['actp'].astype(str))
df['hrmn'] = le.fit_transform(df['hrmn'].astype(str))

In [6]:
df_M = df[df['metropolitan'] == 1]
df_M.head(3)

,num_veh,place,catu,grav,sexe,an_nais,trajet,locp,actp,etatp,...,obsm,choc,manv,date,is_holiday,holiday,secu_used,secu_avail,fatal,year
Num_Acc,,,,,,,,,,,,,,,,,,,,,
201900000001,B01,2.0,2,4,2,2002.0,0.0,-1.0,0,-1.0,...,2.0,5.0,23.0,2019-11-30,False,NaN,3,1,0,2019
201900000001,B01,1.0,1,4,2,1993.0,5.0,-1.0,0,-1.0,...,2.0,5.0,23.0,2019-11-30,False,NaN,3,1,0,2019
201900000001,A01,1.0,1,1,1,1959.0,0.0,-1.0,0,-1.0,...,0.0,3.0,11.0,2019-11-30,False,NaN,3,1,0,2019


In [7]:
#cleanning variables that might not be used in the model
df_rf = df_M.drop(['num_veh','adr','date','holiday','lat', 'long','grav'], axis=1) # grav removed because is similat to target 

In [8]:
 #split the data for modeling
# 2005 to 2019 for trainning  and 2020 to 2021 to test

# Filter data for the training period (2005-2018)
train_data = df_rf[(df_rf['an'] >= 2005) & (df_rf['an'] <= 2019)]

# Filter data for the testing period (2019-2021)
test_data = df_rf[(df_rf['an'] >= 2020) & (df_rf['an'] <= 2021)]
 #Separate features and target variable
X_train = train_data.drop('fatal', axis=1)  # Features for training
y_train = train_data['fatal']  # Target variable for training

# Separate features and target variable
X_test = test_data.drop('fatal', axis=1)  # Features for training
y_test = test_data['fatal']  # Target variable for training


In [9]:
#load model previus saved model

from sklearn.ensemble import RandomForestRegressor
import joblib

# Load the saved RandomForestRegressor model from the file
loaded_rf_model = joblib.load("random_forest_model.pkl")

# Now, you can use loaded_rf_model for predictions
y_pred_loaded = loaded_rf_model.predict(X_test)

In [ ]:
#this is taking to long I will run again at night.
# for now just the upper parts are valid for the model 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Create the RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42)

# Define the extended hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2', 0.2, 0.4]
}

# Increase the number of iterations
n_iter = 10

# Create the randomized search object
randomized_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=n_iter,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=4 ,
    random_state=42
)

# Perform hyperparameter tuning
randomized_search.fit(X_train, y_train)

# Get the best hyperparameter values
best_params = randomized_search.best_params_

: 